In [ ]:
import geopandas as gpd
import shapely.geometry
from shapely import wkt
import pandas as pd
import ast
from shapely.ops import unary_union
import numpy as np

import pandas as pd
import re

In [ ]:
# Geometry loading if it is in `CSV' format

reamining_fields = pd.read_csv("/Users/smit0225/Documents/Oxford/Agriculture/Land_Registry/filtered_fields/new_defra_data/final_database/with_cluster/owner/base_owner/Base_owner_wise_with_geometry.csv",index_col=False)
reamining_fields['geometry'] = reamining_fields['geometry'].apply(wkt.loads)
reamining_fields = gpd.GeoDataFrame(reamining_fields)
reamining_fields = reamining_fields.set_crs(27700, allow_override=True)
reamining_fields

In [ ]:
#final name cleaning code


def conditional_upper(name):
    if name.isupper():
        return name
    else:
        return name.upper()

# Convert to uppercase


def replace_text(x):
    # Replace brackets with spaces
    pattern = r'[\[\](){}<>]'   
    return re.sub(pattern, ' ', x, flags=re.IGNORECASE)

def replace_hyph_text(x):
    # Replace hyphens between letters with spaces
    pattern = r'(?<=[a-zA-Z])-(?=[a-zA-Z])'   
    return re.sub(pattern, ' ', x, flags=re.IGNORECASE)

def replace_and(x):
    # Replace '&' with 'AND'
    return re.sub(r'&', 'AND', x)

def replace_LTD(x):
    # Replace 'LTD' with 'LIMITED'
    return re.sub(r'\bLTD\b', 'LIMITED', x)

def replace_property(x):
    # Replace 'PROPERTY' with 'PROPERTIES'
    return re.sub(r'PROPERTY', 'PROPERTIES', x)  

def remove_space(x):
    # Remove extra spaces
    x = re.sub(r'\s+', ' ', x)
    return x.strip()

def clean_commas(text):
    # Replace commas between words with spaces
    text = re.sub(r'(?<=[a-zA-Z]),(?=[a-zA-Z])', ' ', text)
    # Remove any trailing commas or trailing spaces
    text = re.sub(r',\s*$', '', text)
    # Remove leading or consecutive commas
    text = re.sub(r'(^,|,{2,})', '', text)
    # Remove extra spaces around commas
    text = re.sub(r'\s*,\s*', ' ', text)
    return text.strip()


def replace_special_characters_and_normalize(name):
    # Replace periods and hyphens with spaces
    replaced_chars = re.sub(r'[.-]', ' ', name)
    # Remove leading, trailing, and multiple spaces
    cleaned_name = re.sub(r'\s+', ' ', replaced_chars).strip()
    # Capitalize each word properly
    normalized_name = ' '.join(word.capitalize() for word in cleaned_name.split())
    return normalized_name

def remove_after_limited(x):
    pattern = r'(Limited).*'  # Match "Limited" and everything after
    return re.sub(pattern, r'\1', x, flags=re.IGNORECASE)


# Apply the functions to the 'Owner Name' column


reamining_fields['Owner Name'] = reamining_fields['Owner Name'].apply(conditional_upper) 
reamining_fields["Owner Name"] = reamining_fields["Owner Name"].apply(replace_text)
reamining_fields["Owner Name"] = reamining_fields["Owner Name"].apply(replace_hyph_text)
reamining_fields["Owner Name"] = reamining_fields["Owner Name"].apply(replace_LTD)
reamining_fields["Owner Name"] = reamining_fields["Owner Name"].apply(replace_and)
reamining_fields["Owner Name"] = reamining_fields["Owner Name"].apply(replace_property)
reamining_fields["Owner Name"] = reamining_fields["Owner Name"].apply(replace_special_characters_and_normalize)
reamining_fields["Owner Name"] = reamining_fields["Owner Name"].apply(remove_space)
reamining_fields['Owner Name'] = reamining_fields['Owner Name'].apply(clean_commas)
reamining_fields['Owner Name'] = reamining_fields['Owner Name'].apply(remove_after_limited)
reamining_fields['Owner Name'] = reamining_fields['Owner Name'].str.replace(':', '').str.strip()

reamining_fields['Owner Name'] = reamining_fields['Owner Name'].str.upper()
reamining_fields['Owner Name'] = reamining_fields['Owner Name'].str.title()

